In [1]:
import spotipy
import pandas as pd
import spotipy.util as util
import sys
import numpy as np

In [98]:
scope = 'user-library-read'

client_id = '5c0e610065974491a42d1d8e60d582f7'
client_secret = '116e177b1fe84b1ea3a855aa55ee0a40'
redirect = 'https://localhost/3000'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
print(sys.argv)

token = util.prompt_for_user_token(username, scope, client_id, client_secret,redirect)
sp = spotipy.Spotify(auth=token)


['/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/Users/MikeyMuller/Library/Jupyter/runtime/kernel-c665768d-6e2c-49d4-a156-bef00ce33599.json']


In [93]:
results = sp.search(q='artist:goo goo dolls track:before its', type='track', limit = 50)

items = results.items()

In [94]:
song_uri = str(results['tracks']['items'][0]['uri'])

songs=[]
songs.append(song_uri)
artistUri = results['tracks']['items'][0]['artists'][0]['uri']
print(sp.audio_features(songs))

[{'danceability': 0.295, 'energy': 0.712, 'key': 0, 'loudness': -4.246, 'mode': 1, 'speechiness': 0.0301, 'acousticness': 0.00125, 'instrumentalness': 2.25e-05, 'liveness': 0.224, 'valence': 0.366, 'tempo': 130.139, 'type': 'audio_features', 'id': '4kmGRW1BgoKendL6V7cz29', 'uri': 'spotify:track:4kmGRW1BgoKendL6V7cz29', 'track_href': 'https://api.spotify.com/v1/tracks/4kmGRW1BgoKendL6V7cz29', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4kmGRW1BgoKendL6V7cz29', 'duration_ms': 187760, 'time_signature': 3}]


In [95]:
artist = sp.artist(artistUri)
print(artist['genres'])

['neo mellow', 'permanent wave', 'pop rock', 'post-grunge']


In [96]:
for song in results['tracks']['items']:
    print(song['name'], song['album']['artists'][0]['name'])


Before It's Too Late The Goo Goo Dolls


In [102]:
# Spotify helper functions
def getTrack(trackName, trackArtist):
    trackArtist = trackArtist.lower().split(" with")[0]
    trackArtist = trackArtist.lower().split(" ,")[0]
    trackArtist = trackArtist.split(" and ")[0]
    trackArtist = trackArtist.split(" feat")[0]
    trackArtist = trackArtist.lower().split(" x")[0]
    trackName = trackName.lower().split(" (")[0]
    songs = sp.search(q="artist:"+ trackArtist + " track:" + trackName, type='track', limit = 50)
    if(len(songs['tracks']['items']) == 0):
        print("NOT FOUND")
        return None
    else:
        print("FOUND")
        return songs['tracks']['items'][0]

def getURIs(track):
    artistURI = track['artists'][0]['uri']
    if artistURI is None:
        print("no artist")
    trackURI = track['uri']
    return {'artistURI' : artistURI, 'trackURI' : trackURI}

def getArtist(artistURI):
    return sp.artist(artistURI)

def getArtistGenre(artist):
    return artist['genres']
    
def getTrackFeatures(trackURIs):
    return sp.audio_features(trackURIs)
    

In [42]:
# Dataframe helper functions
def generateColumns(df, columnNames):
    for name in columnNames:
        df[name] = np.nan

def cleanSongNames(df):
    for i, row in df.iterrows():
        df.at[i, 'song'] = df.at[i, 'song'].replace('\'','')
        df.at[i, 'artist'] = df.at[i, 'artist'].replace('\'','')

def fillRow(df, songName, artistName, values):
    for key in values:
        df.loc[(df['song'] == songName) & (df['artist'] == artistName), key] = values[key]

def updateGenre(df, songName, artistName, value):
    df.loc[(df['song'] == songName) & (df['artist'] == artistName), 'genre'] = value



In [283]:
unique_songs = pd.read_csv("/Users/MikeyMuller/Desktop/Junior Year/Semester 1/STOR 320/Final Project/DataSet/UniqueSongs.csv")
cleanSongNames(unique_songs)


In [308]:
all_songs = unique_songs['song'].to_numpy()
all_artists = unique_songs['artist'].to_numpy()
index = 7000
track_URIs = []
for i in range(index,len(all_songs)):
    song_name = str(all_songs[i])
    artist_name = str(all_artists[i])
    track = getTrack(song_name, artist_name)
    if(track is None):
        continue
    URIs = getURIs(track)
    features = getTrackFeatures([URIs['trackURI']])[0]
    fillRow(unique_songs,song_name, artist_name, features)
    artist = getArtist(URIs['artistURI'])
    genre = str(getArtistGenre(artist))
    updateGenre(unique_songs, song_name, artist_name, genre)



future
childish gambino
k. michelle
nicki minaj
ozuna x manuel turizo
Vaina Loca
glee cast
youngboy never broke again
owl city
the pack
chris jamison
eminem
lady gaga
migos
bruno mars
khalid
u2
miranda lambert
panic! at the disco
nipsey hussle
india.arie
lana del rey
beyonce
drake
j. cole
kanye west
coldplay
chevelle
coldplay
marc anthony
chris young
secondhand serenade
quavo
W O R K I N M E
nelly
darius rucker
maroon 5
ying yang twins
ray j
the pussycat dolls
tyga & justin bieber
Wait For A Minute
elliott yamin
brad paisley
daughtry
linkin park
jennifer lopez
seal
john mayer
6ix9ine
shakira
green day
avicii
fetty wap
hilary duff
travis scott
maroon 5
gucci mane x bruno mars x kodak black
Wake Up In The Sky
craig morgan
katy perry
foo fighters
billy currington
kelly clarkson
paula deanda
unk
migos
p!nk
eminem
rich homie quan
jagged edge
the-dream
mary mary
craig david
lonestar
empire of the sun
katy perry
chris brown
50 cent
glee cast
michael jackson with akon
brett eldredge
g-unit
pli

You Dont Know
brantley gilbert
alicia keys
grace
You Dont Own Me
the fray
faith evans
blake lewis
You Give Love A Bad Name
keith urban
amy winehouse
chris cornell
donell jones
t.i.
dj snake & alunageorge
You Know You Like It
nirvana
the band perry
lady antebellum
keith urban
justin moore
jason aldean
avicii
cobra starship
p!nk
nas
josh groban
josh groban & the african childrens choir
You Raise Me Up
michael jackson
marc anthony
kenny chesney
lauren daigle
cole swindell
billie eilish
kelly price
toby keith
lucy hale
dj khaled
rick ross
lil uzi vert
billy ray cyrus
brooks & dunn
hannah montana
george strait
keith urban
vertical horizon
new boyz
james blunt
the offspring
trace adkins
lonestar
mariah carey
keith urban
anita baker
Youre My Everything
taylor swift
empire cast
guerilla black
maria mena
dwayne johnson
kenny chesney
frankie ballard
meek mill
lana del rey
khalid
jay-z + mr. hudson
Young Forever
bruno mars
snoop dogg & wiz khalifa
fabolous
Youngn (Holla Back)
5 seconds of summer


In [310]:
unique_songs.to_csv("unique_songs_with_data.csv")

In [312]:
unique_with_data = unique_songs

In [52]:
unique_songs = pd.read_csv("/Users/MikeyMuller/Desktop/Junior Year/Semester 1/STOR 320/Final Project/DataSet/unique_songs_with_data.csv")


In [103]:
all_songs = unique_songs['song'].to_numpy()
all_artists = unique_songs['artist'].to_numpy()
index = 0
track_URIs = []
for i in range(index,len(all_songs)):
    if not np.isnan(unique_songs.iloc[i]['tempo']):
        continue
    song_name = str(all_songs[i])
    artist_name = str(all_artists[i])
    track = getTrack(song_name, artist_name)
    if(track is None):
        continue
    URIs = getURIs(track)
    features = getTrackFeatures([URIs['trackURI']])[0]
    if(features is None):
        continue
    fillRow(unique_songs,song_name, artist_name, features)
    artist = getArtist(URIs['artistURI'])
    genre = str(getArtistGenre(artist))
    updateGenre(unique_songs, song_name, artist_name, genre)


NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUND
NOT FOUN

In [104]:
unique_songs.query("tempo == 'NaN'")

,Unnamed: 0,Unnamed: 0.1,year,song,artist,peak_pos,weeks_on_chart,instance,danceability,energy,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
38,38,39,2000,15 Minutes,Marc Nelson,27,16,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,75,76,2015,3500,Travi$ Scott Featuring Future & 2 Chainz,82,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,79,80,2012,4 AM,Melanie Fiona,97,8,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,125,126,2007,A Broken Wing,Jordin Sparks,66,2,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,127,128,2009,A Change Is Gonna Come,Adam Lambert,56,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,224,225,2009,Aint No Sunshine,Kris Allen,37,2,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,370,371,2005,And Then What,Young Jeezy Featuring Mannie Fresh,91,11,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,373,374,2013,Angel,Jacquie Lee,87,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,419,420,2009,Apologize,Kris Allen,66,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542,542,543,2001,Bad Boy For Life,"P. Diddy, Black Rob & Mark Curry",70,12,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
unique_songs['tempo'].isna().sum()

212

In [54]:
for i in range(0,10):
    if not np.isnan(unique_songs.iloc[i]['tempo']):
        print("yeet")

yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet


In [106]:
unique_songs.to_csv("unique_songs_with_data.csv")